# Historical Stock Price Returns

### Polygon: https://polygon.io/dashboard

## Setup Environment

In [1]:
# Import libraries

import pandas as pd
import numpy as np

import os
import zipfile
from datetime import datetime, timedelta

In [2]:
# Google BigQuery Authentication

from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Export data

from pandas_gbq import to_gbq
import pickle

# Set your OAuth client ID JSON downloaded from GCP Console
# CLIENT_SECRET_FILE = 'client_secret.json'  # downloaded from GCP
CLIENT_SECRET_FILE = os.path.expanduser("/Users/ryanrunchey/credentials/gcp_credentials/client_secret_295707256455-0fsr3bqoc89psl22fgp2cfipbd4m1s1v.apps.googleusercontent.com.json")
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

# Authenticate interactively (stores a token locally for reuse)
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)
else:
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
    credentials = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(credentials, token)

# Initialize the BigQuery client with those credentials
client = bigquery.Client(credentials=credentials, project="ryanrunchey")

### Test BigQuery Read/Write

In [3]:
# Test query
query = """
SELECT
  underlying_symbol,
  symbol,
  SUM(quantity) AS quantity,
  SUM(fees_and_commissions) AS fees_and_commissions,
  SUM(amount) AS amount,
  SAFE_DIVIDE(SUM(amount), SUM(quantity)) AS net_price_per_unit
FROM
  ryanrunchey.account_transactions.fct_transactions
WHERE
  underlying_symbol = 'ENVX'
GROUP BY
  1,2
ORDER BY
  1,2
"""

df = client.query(query).to_dataframe()
df

,underlying_symbol,symbol,quantity,fees_and_commissions,amount,net_price_per_unit
0,ENVX,-ENVX231117C15,0E-9,2.720000000,345.280000000,None
1,ENVX,-ENVX240119C10,0E-9,2.680000000,-152.680000000,None
2,ENVX,-ENVX240119C17.5,0E-9,2.730000000,529.190000000,None
3,ENVX,-ENVX240119C20,0E-9,1.360000000,258.600000000,None
4,ENVX,-ENVX240719C15,0E-9,4.020000000,-169.020000000,None
5,ENVX,-ENVX250117C12.5,0E-9,6.740000000,3538.990000000,None
6,ENVX,-ENVX250117C20,0E-9,10.090000000,3348.510000000,None
7,ENVX,-ENVX260116C8,-36.000000000,24.500000000,13257.500000000,-368.263888889
8,ENVX,ENVX,6407.637000000,0.220000000,-58547.820000000,-9.137193633
9,ENVX,ENVX 01/19/2024 9.00 C,0E-9,5.290000000,-901.290000000,None


## Import Data

In [4]:
query = """SELECT * FROM ryanrunchey.historical_stock_price_returns.polygon_historical_stock_prices"""

df_import = client.query(query).to_dataframe()
df_import.rename(columns={"ticker": "symbol"}, inplace=True)
df_import.head()

,exchange,symbol,date,close
0,XASE,KRY,2006-09-15,3.1300
1,XASE,VUG,2006-09-15,54.2900
2,XASE,BHM,2006-09-15,10.9005
3,XASE,BGF,2006-09-15,18.9500
4,XASE,EIF,2006-09-15,14.2400


In [5]:
df_import.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85859 entries, 0 to 85858
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   exchange  85859 non-null  object 
 1   symbol    85859 non-null  object 
 2   date      85859 non-null  object 
 3   close     85859 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.6+ MB


In [6]:
# Format, clean, and sort dataframe

df_import['date'] = pd.to_datetime(df_import['date'])
df_import.sort_values(['exchange', 'symbol', 'date'], inplace=True)
df_import.reset_index(drop=True, inplace=True)
df_import.head()

,exchange,symbol,date,close
0,XASE,AAB.WS,2006-09-15,6.94
1,XASE,AAB.WS,2007-01-16,5.25
2,XASE,AAB.WS,2007-09-14,3.74
3,XASE,AAB.WS,2008-01-15,7.23
4,XASE,AAC,2003-09-17,122.40


## Pivot Data

In [7]:
# Pivot to count # of stocks less than X price by exchange

list_dates = ['2002-09-16', '2003-09-15', '2004-09-14', '2005-09-16', '2006-09-15', '2007-09-14', '2008-09-15', \
              '2009-09-14', '2010-09-14', '2011-09-16', '2012-09-14', '2013-09-16', '2014-09-15', '2015-09-14', \
              '2016-09-16', '2017-09-15', '2018-09-14', '2019-09-14', '2020-09-14', '2021-09-14', '2022-09-14']

pd_list_dates = pd.to_datetime(list_dates)  # ensure correct dtype
mask = df_import['date'].isin(pd_list_dates)

df_count_equities = df_import[mask].copy(deep=True)
df_count_equities['close_under_3'] = (df_count_equities['close'] <= 3)
df_count_equities['close_under_2'] = (df_count_equities['close'] <= 2)
df_count_equities['close_under_1'] = (df_count_equities['close'] <= 1)
df_count_equities['close_under_0_50'] = (df_count_equities['close'] <= 0.5)
df_count_equities_agg = df_count_equities.groupby(['exchange', 'date']).agg({
    'symbol':'count', 
    'close_under_3':'sum',
    'close_under_2':'sum',
    'close_under_1':'sum',
    'close_under_0_50':'sum',
})
df_count_equities_agg

symbol  close_under_3  close_under_2  close_under_1  \
exchange date                                                              
XASE     2005-09-16    1043            177            114             71   
         2006-09-15    1048            201            132             67   
         2007-09-14    1243            223            165             95   
         2011-09-16     480            149            106             49   
         2012-09-14     447            141            108             58   
         2016-09-16     328             93             72             44   
         2017-09-15     339             94             74             44   
         2018-09-14     333             90             65             33   
XNAS     2011-09-16      95              7              2              1   
         2012-09-14      82              4              2              1   
         2016-09-16    2178            230            138             44   
         2017-09-15    3099            341            240            114   
         2018-09-14    3191            317            218            101   

                     close_under_0_50  
exchange date                          
XASE     2005-09-16                30  
         2006-09-15                33  
         2007-09-14                33  
         2011-09-16                22  
         2012-09-14                32  
         2016-09-16                21  
         2017-09-15                22  
         2018-09-14                15  
XNAS     2011-09-16                 1  
         2012-09-14                 1  
         2016-09-16                20  
         2017-09-15                58  
         2018-09-14                57

In [8]:
# Export to csv
df_count_equities_agg.to_csv("polygon_pivot_count_of_stocks_priced_less_than_x.csv")

# Export to pickle
df_count_equities_agg.to_pickle("polygon_pivot_count_of_stocks_priced_less_than_x.pkl")

# Export to BigQuery
to_gbq(
    dataframe=df_count_equities_agg,
    destination_table="historical_stock_price_returns.polygon_pivot_count_of_stocks_priced_less_than_x",
    project_id="ryanrunchey",
    if_exists="replace"  # or "append"
)

100%|██████████| 1/1 [00:00<00:00, 6887.20it/s]


## Price Return Calculations

In [9]:
# Import historical stock dates
query = """SELECT * FROM ryanrunchey.historical_stock_price_returns.historical_stock_dates"""

df_dates = client.query(query).to_dataframe()
df_dates

,start_date,end_date
0,2002-09-18,2005-01-18
1,2003-09-17,2006-01-17
2,2004-09-15,2007-01-16
3,2005-09-16,2008-01-15
4,2006-09-15,2009-01-13
5,2007-09-14,2010-01-12
6,2008-09-17,2011-01-18
7,2009-09-16,2012-01-17
8,2010-09-15,2013-01-14
9,2011-09-16,2014-01-14


In [10]:
# Example structure
# df_dates: columns = ['start_date', 'end_date']
# df_import: columns = ['exchange', 'symbol', 'date', 'close', ...]

# Ensure date columns are datetime
df_dates['start_date'] = pd.to_datetime(df_dates['start_date'])
df_dates['end_date'] = pd.to_datetime(df_dates['end_date'])
df_import['date'] = pd.to_datetime(df_import['date'])

# Result holder
joined_rows = []

# Iterate through each date pair
for _, row in df_dates.iterrows():
    start = row['start_date']
    end = row['end_date']
    
    # Filter df_import for start and end dates
    df_start = df_import[df_import['date'] == start].copy()
    df_end = df_import[df_import['date'] == end].copy()

    # Rename columns for joining clarity
    df_start = df_start.rename(columns={
        'date': 'start_date',
        'close': 'start_close'  # adjust based on your needs
    })
    df_end = df_end.rename(columns={
        'date': 'end_date',
        'close': 'end_close'
    })

    # Join on exchange + symbol
    joined = pd.merge(
        df_start,
        df_end,
        on=['exchange', 'symbol'],
        how='left'
    )

    # Always keep the current start_date; only match end_date when it exists
    joined = joined[
        (joined['start_date'] == start) &
        ((joined['end_date'] == end) | (joined['end_date'].isna()))
    ]

    joined_rows.append(joined)

# Concatenate all pairwise results
df_joined = pd.concat(joined_rows, ignore_index=True)

# Preview
df_joined.head()

,exchange,symbol,start_date,start_close,end_date,end_close
0,XASE,AAC,2003-09-17,122.40,2006-01-17,81.60
1,XASE,AAp,2003-09-17,80.50,NaT,NaN
2,XASE,AB,2003-09-17,1.35,NaT,NaN
3,XASE,ABL,2003-09-17,6.80,2006-01-17,9.30
4,XASE,ABP,2003-09-17,0.99,2006-01-17,6.18


In [11]:
# Calculate returns
df_joined['price_difference'] = df_joined['end_close'] - df_joined['start_close']
df_joined['roi_pct'] = np.where(
    (df_joined['start_close'].notna()) & 
    (df_joined['end_close'].notna()) & 
    (df_joined['start_close'] != 0),
    (df_joined['end_close'] - df_joined['start_close']) / df_joined['start_close'],
    np.nan
)

df_joined.head()

,exchange,symbol,start_date,start_close,end_date,end_close,price_difference,roi_pct
0,XASE,AAC,2003-09-17,122.40,2006-01-17,81.60,-40.80,-0.333333
1,XASE,AAp,2003-09-17,80.50,NaT,NaN,NaN,NaN
2,XASE,AB,2003-09-17,1.35,NaT,NaN,NaN,NaN
3,XASE,ABL,2003-09-17,6.80,2006-01-17,9.30,2.50,0.367647
4,XASE,ABP,2003-09-17,0.99,2006-01-17,6.18,5.19,5.242424


In [12]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37664 entries, 0 to 37663
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   exchange          37664 non-null  object        
 1   symbol            37664 non-null  object        
 2   start_date        37664 non-null  datetime64[ns]
 3   start_close       37664 non-null  float64       
 4   end_date          26095 non-null  datetime64[ns]
 5   end_close         26095 non-null  float64       
 6   price_difference  26095 non-null  float64       
 7   roi_pct           26095 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 2.3+ MB


In [13]:
mask = df_joined['end_close'].isna()
df_joined[mask]

,exchange,symbol,start_date,start_close,end_date,end_close,price_difference,roi_pct
1,XASE,AAp,2003-09-17,80.5000,NaT,NaN,NaN,NaN
2,XASE,AB,2003-09-17,1.3500,NaT,NaN,NaN,NaN
5,XASE,ACMpA,2003-09-17,9.0500,NaT,NaN,NaN,NaN
11,XASE,ADY,2003-09-17,40.2100,NaT,NaN,NaN,NaN
15,XASE,AGD,2003-09-17,1.7000,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
37658,XNAS,ZVO,2021-09-16,2.4100,NaT,NaN,NaN,NaN
37659,XNAS,ZWRKU,2021-09-16,9.8854,NaT,NaN,NaN,NaN
37660,XNAS,ZWRKW,2021-09-16,0.6100,NaT,NaN,NaN,NaN
37661,XNAS,ZY,2021-09-16,13.7500,NaT,NaN,NaN,NaN


In [14]:
# Check for stocks that have no end_close price
start_tickers = set(df_start['symbol'])
end_tickers = set(df_end['symbol'])
missing = start_tickers - end_tickers
print(f"Start symbols: {len(start_tickers)}")
print(f"End symbols: {len(end_tickers)}")
print(f"Missing from end: {len(missing)}")

print()

df_joined.isna().sum()

Missing from end: 1633



exchange                0
symbol                  0
start_date              0
start_close             0
end_date            11569
end_close           11569
price_difference    11569
roi_pct             11569
dtype: int64

In [15]:
# Export to pickle
df_joined.to_pickle("polygon_stock_price_returns_851_trading_days.pkl")

# Export to BigQuery
to_gbq(
    dataframe=df_joined,
    destination_table="historical_stock_price_returns.polygon_stock_price_returns_851_trading_days",
    project_id="ryanrunchey",
    if_exists="replace"  # or "append"
)

100%|██████████| 1/1 [00:00<00:00, 6657.63it/s]
